In [2]:
import numpy as np
from torch.utils.data import Dataset
import pydicom as dicom
import os
import cv2
import matplotlib.pyplot as plt

In [45]:
def concat(data1,data2):
    #dicom 与 seg 合并程序
    data = np.concatenate((data1[np.newaxis,:],data2[np.newaxis,:]),axis = 0)
    return data

In [47]:
def dcm(list):
    #在slices 路径下仅保存 .dcm
    list_ = []
    for i in list:        
        if i.split('.')[-1]=='dcm':
           list_.append(i)
        
    return list_
        

In [48]:
def overlay(path):
    # 输入：病例目录  ：slices  seg     
    # 输出：3D文件  2, 256, 512, 512
    data = []
    label = path.split('/')[-2]
    path1 = os.path.join(path,'slices')
    path2 = os.path.join(path,'seg')
    dir = D(os.listdir(path1))
    
    for dcm_img_id in dcm(dir):
        dicom_path = os.path.join(path1,dcm_img_id)
        dcm_img = dicom.read_file(dicom_path).pixel_array
        seg_path = os.path.join(path2,dcm_img_id.split('.')[-2]+'.png')
        
        seg_img = cv2.imread(seg_path,0)
#         print seg_img.shape
        data.append(concat(seg_img,dcm_img))
    data = np.array(data)
    data = data.transpose(1,0,2,3)
    return data.shape,label          # 返回叠加以后的数据和label
overlay('./data/0/3')
    
    

((2, 256, 512, 512), '0')

In [49]:
def D(list):
    # 去掉mac中的Ds
    if '.DS_Store' in list:
            list.remove('.DS_Store')
    return list

In [50]:
def default_loader(img_id):
#   输入文件的ID路径
#   输出3d文件和label
    print img_id    
    img,label = overlay(img_id)
    return img,label

In [51]:
# torch_dataset = Data.TensorDataset(data_tensor=x, target_tensor=y)
class myImageFloder(Dataset):
    
    def __init__(self,root,transform = None,target_transform=None,loader = default_loader):
        self.root = root       
        self.tranform = transform
        self.target_transform = target_transform
        self.loader = loader
        class_ = os.listdir(root)  # root 文件下的0，1文件
        class_ = D(class_)
        img_id = []
        
        for i in class_:
            for j in D(os.listdir(os.path.join(root,i))):
                path = os.path.join(os.path.join(root,i),j)
                img_id.append(path)     
                
        self.img_id = img_id
        
        
            
    def __getitem__(self,index):
        img_id = self.img_id[index]
        
        img,label = self.loader(img_id)
        
        return img,label
        
    
    def __len__(self):
        return len(self.imgs)

In [58]:
def testmyImageFloder(root='./data1'):
    dataloader = myImageFloder(root)
    
    for index , (img, label) in enumerate(dataloader):
        print index
        print (img, 'label',label)
       
testmyImageFloder('./data1')

./data1/0/P00017269T20180702R084831
0
((2, 224, 512, 512), 'label', '0')
./data1/0/P00012233T20180622R084430
1
((2, 224, 512, 512), 'label', '0')
./data1/0/P00007786T20180621R182848
2
((2, 224, 512, 512), 'label', '0')
./data1/0/P00012690T20180613R094036
3
((2, 256, 512, 512), 'label', '0')
./data1/0/P00014395T20180608R133636
4
((2, 224, 512, 512), 'label', '0')
./data1/0/P00018129T20180702R160436
5
((2, 192, 512, 512), 'label', '0')
./data1/0/P00016611T20180607R160455
6
((2, 256, 512, 512), 'label', '0')
./data1/1/P00393459T20180612R110836
7
((2, 256, 512, 512), 'label', '1')
./data1/1/P00442249T20180522R155657
8
((2, 224, 512, 512), 'label', '1')
./data1/1/P00426290T20180625R155657
9
((2, 224, 512, 512), 'label', '1')
./data1/1/P00410115T20180604R173634
10
((2, 224, 512, 512), 'label', '1')
./data1/1/P00435379T20180625R151240
11
((2, 256, 512, 512), 'label', '1')
./data1/1/P00443879T20180522R154453
12
((2, 256, 512, 512), 'label', '1')
./data1/1/P00379300T20180709R111234
13
((2, 192,